In [14]:
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.loaders import SpontaneousActivityLoader
from drn_interactions.interactions.pairwise import PairwiseCorr
from drn_interactions.io import load_neurons
import seaborn as sns
import pandas as pd
import numpy as np
from drn_interactions.interactions.graph_clustering import SpectralCluster, df_to_graph, ClusterRunner, ClusterEvaluation
import warnings
from drn_interactions.config import Config



In [10]:

def reorder_df_affinity(df, ensemble_labels):
    idx = np.argsort(ensemble_labels)
    return df.iloc[idx, idx]

In [11]:
sessions = load_neurons()["session_name"].unique()
session = sessions[9]

loader = SpontaneousActivityLoader(session_name=session, shuffle=False)
preprocessor = InteractionsPreprocessor(z=True, minmax=False)
corr = PairwiseCorr(
    remove_self_interactions=True, 
    rectify=True, 
    shuffle=False, 
    remove_duplicate_combs=True,
    )
evaluator = ClusterEvaluation()

cluster = SpectralCluster(
    n_clusters=np.arange(2, 8), 
    spectral_kws={"affinity": "precomputed"},
    n_init=15,
    )
runner = ClusterRunner(
    loader=loader,
    preprocessor=preprocessor,
    affinity_calculator=corr,
    clusterer=cluster,
    evalulator=evaluator,
)

In [13]:
ensemble_frames = []
ensemble_stats_frames = []
current_max = 1

for session in sessions:
    runner.loader.session_name = session
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        _, ensembles, _, ensemble_stats = runner.run(20)
    ensembles["ensemble"] = ensembles["ensemble"] + current_max
    ensemble_stats["ensemble"] = ensemble_stats["ensemble"] + current_max
    ensemble_stats_frames.append(ensemble_stats.assign(session_name=session))
    ensemble_frames.append(ensembles.assign(session_name=session))
    current_max = ensembles["ensemble"].abs().max() + 1

df_ensembles = pd.concat(ensemble_frames).reset_index(drop=True)
df_ensembles_stats = pd.concat(ensemble_stats_frames).reset_index(drop=True)

In [15]:
dd = Config.derived_data_dir / "ensembles"
dd.mkdir(exist_ok=True)
df_ensembles.to_csv(dd / "spont - ensembles.csv", index=False)
df_ensembles.to_csv(dd / "spont - stats.csv", index=False)